In [78]:
import sys
import os
import numpy as np
import skaero
from skaero.atmosphere import coesa

print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
print('Python version: ' + sys.version)
print('Numpy version: ' + np.version.version)
print('Scikit-Aero version: ' + skaero.__version__)
print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Python version: 3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)]
Numpy version: 1.16.3
Scikit-Aero version: 0.1
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [12]:
h, T, p, rho = coesa.table(1000)

print('At {} m, the temperature is {} K, the ambient pressure is {:.2f} Pa, and the density is {:.3} kg/m3.'.format(h, T, p, rho))

At 1000 m, the temperature is 281.65 K, the ambient pressure is 89874.57 Pa, and the density is 1.11 kg/m3.


In [38]:
# Dynamic Pressure on an object

# Velocity, in m/s
u = 50

# Altitude, in m
h = 1000

# Density and Temperature calculated based on the US Standard Atmosphere
h, T, p, rho = coesa.table(h)

# Dynamic Pressure, in Pa (N*m^-2)
q = 0.5*rho*u**2

In [39]:
h = np.array([10, 100, 1000, 10000])

h, T, p, rho = coesa.table(h)
print(rho)

q = 0.5*rho*u**2

for alt,den in zip(h,q):
    print('At {:.0f}m, dynamic pressure is {:.2E}Pa.'.format(alt, den))

[1.22382355 1.21328194 1.11164181 0.41270625]
At 10m, dynamic pressure is 1.53E+03Pa.
At 100m, dynamic pressure is 1.52E+03Pa.
At 1000m, dynamic pressure is 1.39E+03Pa.
At 10000m, dynamic pressure is 5.16E+02Pa.


---

### Loads Functions

Using the basic principles above to extract dynamic pressures, load cards can be written for specific Nastran elements.

In [75]:
def getEID(cardType, file):
    '''
    This function reads a Nastran fem file, and returns a list of all element ids, for the selected type.
    The cardType must be a string of the appropriate Nastran element type (eg. 'cquad4', 'CTRIA6').
    Case does not matter.
    The function assumes (and only works with) short-field format for Nastran.
    Output is a list of EID's (integers).
    '''
    # Open the source file
    f = open(file, 'r')
    # Initate the output
    EID = []
    
    # Read the lines into a list
    lines = f.readlines()
    
    # Loop over each line
    for L in lines:
        # If the first Nastran field is the card we want:
        if L[:8].upper() == cardType.upper().ljust(8):
            # append the text in the ID field as an integer to the output
            EID.append(int(L[8:16].rstrip()))
            
    return EID


def makePressures(S, q, EID, x, y, z, filename='loads.out'):
    '''
    This function writes out applied pressures to Nastran elements. The specific bulk card used is a PLOAD4.
    The global coordinate system is used, and the orientation vector can be defined.
    The pressure is applied as a constant over each entire element (no gradient over a single element).
    The function assumes (and only works with) short-field format for Nastran.
    Input is the set ID, a dynamic pressure, list of element ids, orientation vector components,
    and an output filename (defaulted to 'loads.out').
    Output is a text file with all the PLOAD4 cards.
    
    Returns nothing, file is written.
    '''
    # Create the output file
    f = open(filename, 'w+')
    # Loop over element id
    for e in EID:
        # Write both lines of the PLOAD card.
        # See MSC Nastran quick-reference guide on PLOAD4 card for details on fields
        f.write('PLOAD4  {:<8d}{:<8d}{:<8.3E}\n                {:<8d}{:<8d}{:<8d}\n'.format(S,e,q,x,y,z) )
        

